In [ ]:
!pip install requests
!pip install selenium
!pip install numpy
!pip install pandas

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import requests
import time

In [ ]:
url = "https://eswchildcare.ca/"

driver = webdriver.Firefox(executable_path=r'C:\geckodriver.exe')
driver.get(url)
time.sleep(10)
page_num = 0

while page_num < 23:
    driver.find_element_by_css_selector('#btnShowMore').click()
    page_num += 1
    print("getting page number "+str(page_num))
    time.sleep(15)
    page_source=driver.page_source

In [ ]:
soup = BeautifulSoup(page_source,'html.parser')

In [ ]:
remaining_vaancies_list = []
age_0_to_2 = []
age_2_to_6 = []
age_6_to_12 = []


y = soup.findAll('div',{"class":"col-4"})

for j in y:
    xx = j.find('div',{'class':"display-5"}).text
    remaining_vaancies_list.append(xx)
    

age_0_to_2 = remaining_vaancies_list[0:1617:3]
age_2_to_6 = remaining_vaancies_list[1:1617:3]
age_6_to_12 = remaining_vaancies_list[2:1617:3]

In [ ]:
facility_name = []
map_link = []
facility_type = []
facility_area = []
facility_phone = []
facility_email = []
facility_address = []
facility_region = []
facility_area = []
facility_neighbourhood = []
weekdays = []
weekends = []
evenings = []
overnights = []

item = soup.findAll('div',{"class":"col-lg-6 col-sm-12 mb-4"})

for i in item:
    
    child_care_facilities = i.find('h4',{"class":"child-care-facility-name"}).text
    facility_name.append(child_care_facilities)
        
    try:
        facility_map_links = i.find('a',{"class":"text-light"}).get('href')
        map_link.append(facility_map_links)
    except:
        map_link.append('Map link Not found')
        
    facility_types = i.find('span',{"class":"child-care-facility-type"}).text
    facility_type.append(facility_types)

        
    facility_areas = i.find('span',{"class":"child-care-area-footer"}).text
    facility_area.append(facility_areas)

    
    try:
        facility_phones = i.find('a',{"class":"child-care-facility-phone-number"}).text
        facility_phone.append(facility_phones)
    except:
        facility_phone.append('Phone Number Not found')
        
    
    try:
        facility_emails = i.find('a',{"class":"child-care-facility-email"}).text
        facility_email.append(facility_emails)
    except:
        facility_email.append('E-mail Not found')
        
    try:
        facility_addresses = i.find('address',{"class":"child-care-facility-address"}).text
        facility_address.append(facility_addresses)
    except:
        facility_address.append('Address Not found')
    
    try:
        facility_regions = i.find('span',{"class":"child-care-facility-region"}).text
        facility_region.append(facility_regions)
    except:
        facility_region.append('Region Not found')
    

    try:
        facility_neighbourhoods = i.find('span',{"class":"child-care-facility-neighbourhood"}).text
        facility_neighbourhood.append(facility_areas)
    except:
        facility_neighbourhood.append('Neighbourhood Not found')


    try:    
        a = i.find('div',{"class":"extra-content"})

        checks=a.findAll('div',{"class":"display-5"})

        for j in checks:


            if j['aria-label'] == 'Open Weekdays':
                j=str(j)
                if("fa fa-check") in j:
                    weekdays.append("Yes")
                elif "fa fa-times" in j:
                    weekdays.append("No")

            elif j['aria-label'] == 'Open Weekends':
                j=str(j)
                if("fa fa-check") in j:
                    weekends.append("Yes")
                elif "fa fa-times" in j:
                    weekends.append("No")

            elif j['aria-label'] == 'Open Evenings':
                j=str(j)
                if("fa fa-check") in j:
                    evenings.append("Yes")
                elif "fa fa-times" in j:
                    evenings.append("No")

            elif j['aria-label'] == 'Open Overnight':
                j=str(j)
                if("fa fa-check") in j:
                    overnights.append("Yes")
                elif "fa fa-times" in j:
                    overnights.append("No")

                            
    except:
        weekdays.append("Not found")
        weekends.append("Not found")
        evenings.append("Not found")
        overnights.append("Not found")


In [ ]:

child_care_dict = {
    
    'Child Care Facility' : facility_name ,
    'Map Link' : map_link,
    'Type' : facility_type ,
    'Phone Number' : facility_phone ,
    'Email' : facility_email,
    'Address' : facility_address,
    'Region' : facility_region,
    'Area' : facility_area,
    'Neighbourhood': facility_neighbourhood, 
    'Vacancy in Age 0 to 2' : age_0_to_2 ,
    'Vacancy in Age 2 to 6' : age_2_to_6,
    'Vacancy in Age 6 to 12' : age_6_to_12,
    'Open Weekdays' : weekdays,
    'Open Weekends' : weekends,
    'Open Evenings' : evenings,
    'Open Overnight' : overnights,
}

In [ ]:
df = pd.DataFrame.from_dict(child_care_dict,orient='index').T

In [ ]:
df = df[0:538]

In [ ]:
df

In [ ]:
df.to_pickle("Manitoba_Child_Care_Facility.pkl")